In [2]:
pip install flask psycopg2 geopandas geojson

     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     -- ------------------------------------- 0.1/1.2 MB 2.0 MB/s eta 0:00:01
     ------------ --------------------------- 0.4/1.2 MB 4.5 MB/s eta 0:00:01
     ---------------------- ----------------- 0.6/1.2 MB 5.1 MB/s eta 0:00:01
     --------------------------------- ------ 1.0/1.2 MB 5.5 MB/s eta 0:00:01
     ---------------------------------------- 1.2/1.2 MB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from flask import Flask, jsonify, request
import psycopg2
from flask_cors import CORS
from shapely.wkt import loads
from geojson import FeatureCollection, Feature
import json
from pyproj import Transformer
from sklearn.cluster import KMeans
import numpy as np
import requests

app = Flask(__name__)
CORS(app)

# Database connection
conn = psycopg2.connect(
    dbname="emergency_db",
    user="postgres",
    password="password",  # Replace with your actual password
    host="localhost",
    port="35432"  # Default PostgreSQL port
)

# Define the transformer to convert from UTM (32632) to WGS84 (4326)
transformer = Transformer.from_crs("EPSG:32632", "EPSG:4326", always_xy=True)

@app.route('/')
def home():
    return "Welcome to the Emergency Response System!"

@app.route('/api/accidents')
def get_accidents():
    try:
        category = request.args.get('category', default=None, type=int)
        cur = conn.cursor()
        query = 'SELECT id, ST_AsText(geom) AS geom, "UKATEGORIE" FROM accidents'
        if category is not None:
            query += f' WHERE "UKATEGORIE" = {category};'
        else:
            query += ';'
        cur.execute(query)
        results = cur.fetchall()
        cur.close()

        # Map numerical categories to descriptive names
        category_map = {
            1: "Accident with fatalities",
            2: "Accident with serious injuries",
            3: "Accident with minor injuries"
        }

        # Convert WKT to Shapely geometries and create GeoJSON features
        features = []
        for id, geom_wkt, category in results:
            geom = loads(geom_wkt)  # Convert WKT to Shapely geometry
            feature = Feature(geometry=geom, properties={"id": id, "category": category_map.get(category, "Unknown")})
            features.append(feature)

        # Wrap features in a FeatureCollection
        feature_collection = FeatureCollection(features)
        return jsonify(feature_collection)
    except Exception as e:
        conn.rollback()  # Roll back the transaction
        return jsonify({"error": str(e)}), 500

@app.route('/api/optimize')
def optimize_locations():
    try:
        num_clusters = request.args.get('num_clusters', default=5, type=int)
        cur = conn.cursor()
        query = """
            SELECT 
                ST_X(ST_Transform(a.geom, 4326)) AS longitude, 
                ST_Y(ST_Transform(a.geom, 4326)) AS latitude, 
                p.density
            FROM accidents a
            JOIN population_density p ON ST_Intersects(a.geom, p.geom)
            JOIN landuse l ON ST_Intersects(a.geom, l.geom)
            WHERE a."UKATEGORIE" IN (1, 2)  -- Focus on high-risk accidents
            AND l.fclass IN ('residential', 'commercial', 'industrial');  -- Include only suitable land use types
        """
        cur.execute(query)
        results = cur.fetchall()
        cur.close()

        # Convert results to a numpy array for clustering
        data = np.array([(lon, lat) for lon, lat, _ in results])
        weights = np.array([density for _, _, density in results])

        # Use k-means clustering with weights
        kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(data, sample_weight=weights)
        optimal_locations = kmeans.cluster_centers_

        # Convert optimal locations to GeoJSON format
        optimal_locations_geojson = [
            {"type": "Point", "coordinates": [float(lon), float(lat)]}
            for lon, lat in optimal_locations
        ]

        return jsonify({"optimal_locations": optimal_locations_geojson})
    except Exception as e:
        conn.rollback()  # Roll back the transaction
        return jsonify({"error": str(e)}), 500

@app.route('/api/accidents_by_landuse')
def get_accidents_by_landuse():
    try:
        cur = conn.cursor()
        query = """
            SELECT 
                l.fclass AS landuse_type, 
                COUNT(a.id) AS accident_count
            FROM landuse l
            JOIN accidents a ON ST_Intersects(l.geom, a.geom)
            GROUP BY l.fclass
            ORDER BY accident_count DESC;
        """
        cur.execute(query)
        results = cur.fetchall()
        cur.close()

        # Convert results to a list of dictionaries
        accidents_by_landuse = [{"landuse_type": landuse_type, "accident_count": count} for landuse_type, count in results]

        return jsonify(accidents_by_landuse)
    except Exception as e:
        conn.rollback()  # Roll back the transaction
        return jsonify({"error": str(e)}), 500

@app.route('/api/landuse')
def get_landuse():
    try:
        cur = conn.cursor()
        query = """
            SELECT 
                ST_AsGeoJSON(ST_Transform(geom, 4326)) AS geometry, 
                fclass  -- Correct column name
            FROM landuse;
        """
        cur.execute(query)
        results = cur.fetchall()
        cur.close()

        # Convert results to GeoJSON format
        features = []
        for geometry, fclass in results:
            geom = json.loads(geometry)
            features.append({
                "type": "Feature",
                "geometry": geom,
                "properties": {"type": fclass}  # Use the correct property name here
            })

        return jsonify({"type": "FeatureCollection", "features": features})
    except Exception as e:
        conn.rollback()  # Roll back the transaction
        return jsonify({"error": str(e)}), 500

@app.route('/api/population_density')
def get_population_density():
    try:
        cur = conn.cursor()
        query = """
            SELECT 
                ST_AsGeoJSON(ST_Transform(geom, 4326)) AS geometry, 
                density
            FROM population_density;
        """
        cur.execute(query)
        results = cur.fetchall()
        cur.close()

        # Convert results to GeoJSON format
        features = []
        for geometry, density in results:
            geom = json.loads(geometry)
            features.append({
                "type": "Feature",
                "geometry": geom,
                "properties": {"density": density}
            })

        return jsonify({"type": "FeatureCollection", "features": features})
    except Exception as e:
        conn.rollback()  # Roll back the transaction
        return jsonify({"error": str(e)}), 500

@app.route('/api/roads')
def get_roads():
    try:
        cur = conn.cursor()
        query = """
            SELECT 
                ST_AsGeoJSON(ST_Transform(geom, 4326)) AS geometry, 
                name
            FROM roads;
        """
        cur.execute(query)
        results = cur.fetchall()
        cur.close()

        # Convert results to GeoJSON format
        features = []
        for geometry, name in results:
            geom = json.loads(geometry)
            features.append({
                "type": "Feature",
                "geometry": geom,
                "properties": {"name": name}
            })

        return jsonify({"type": "FeatureCollection", "features": features})
    except Exception as e:
        conn.rollback()  # Roll back the transaction
        return jsonify({"error": str(e)}), 500

@app.route('/api/route')
def get_route():
    try:
        # Get start and end coordinates from query parameters
        start_lon = request.args.get('start_lon', type=float)
        start_lat = request.args.get('start_lat', type=float)
        end_lon = request.args.get('end_lon', type=float)
        end_lat = request.args.get('end_lat', type=float)

        # Validate inputs
        if None in [start_lon, start_lat, end_lon, end_lat]:
            return jsonify({"error": "Missing start or end coordinates"}), 400

        # OSRM API URL (public OSRM demo server)
        osrm_url = f"http://router.project-osrm.org/route/v1/driving/{start_lon},{start_lat};{end_lon},{end_lat}?overview=full&geometries=geojson"

        # Fetch route from OSRM
        response = requests.get(osrm_url)
        if response.status_code != 200:
            return jsonify({"error": "Failed to fetch route from OSRM"}), 500

        # Parse the route data
        route_data = response.json()
        route_geometry = route_data['routes'][0]['geometry']

        # Return the route geometry in GeoJSON format
        return jsonify({
            "type": "Feature",
            "geometry": route_geometry,
            "properties": {
                "distance": route_data['routes'][0]['distance'],  # Distance in meters
                "duration": route_data['routes'][0]['duration']   # Duration in seconds
            }
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\User\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
